In [1]:
import pandas as pd
from tqdm.notebook import tqdm 
import gc
import numpy as np

In [2]:
import statistics as stat
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda

In [3]:
path = '../data'
y_train = pd.read_csv(f'{path}/y_train.csv', index_col = 'key_value')

## Procesar RCC

In [4]:
rcc_train = pd.read_csv(f'{path}/rcc_train.csv')
rcc_test = pd.read_csv(f'{path}/rcc_test.csv')

In [5]:
rcc_test['cod_instit_financiera'].fillna(rcc_test['cod_instit_financiera'].value_counts().index[0], inplace=True)
rcc_test['PRODUCTO'].fillna(rcc_test['PRODUCTO'].value_counts().index[0], inplace=True)

In [6]:
dict_ = {'codmes': 'int32',
 'key_value': 'int32',
 'condicion': 'int32',
 'tipo_credito': 'int32',
 'cod_instit_financiera': 'int32',
 'PRODUCTO': 'int32',
 'RIESGO_DIRECTO': 'int32',
 'COD_CLASIFICACION_DEUDOR': 'int32'}
rcc_train = rcc_train.astype(dict_)
rcc_test = rcc_test.astype(dict_)

In [7]:
bins = [-1, 0, 10, 20, 30, 60, 90, 180, 360, 720, float("inf")]
rcc_train["condicion"] = pd.cut(rcc_train.condicion, bins)
rcc_train["condicion"] = rcc_train["condicion"].cat.codes
rcc_test["condicion"] = pd.cut(rcc_test.condicion, bins)
rcc_test["condicion"] = rcc_test["condicion"].cat.codes

In [8]:
# moda=lambda x: calculate_mode(x)
# moda.__name__='mode'
# agg_rcc = {'cod_instit_financiera':['nunique','min','max',moda],
#            'PRODUCTO':['nunique','min','max',moda],}

In [9]:
list_rcc_train_agg = []
for n,i in enumerate([sorted(set(rcc_train.codmes),reverse=True)[-1]]):
    for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR','condicion','cod_instit_financiera','PRODUCTO']:
        print(f'haciendo {c} desde {i}')
        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=np.nan)
        rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldoSum_ult{n+1}meses' for c in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()

        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.size().unstack(level = 1, fill_value=np.nan)
        rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldoUnique_ult{n+1}meses' for c in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()

        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.min().unstack(level = 1, fill_value=np.nan)
        rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldoMin_ult{n+1}meses' for c in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()

        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.max().unstack(level = 1, fill_value=np.nan)
        rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldoMax_ult{n+1}meses' for c in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()

        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.std().unstack(level = 1, fill_value=np.nan)
        rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldoStd_ult{n+1}meses' for c in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()

        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.mean().unstack(level = 1, fill_value=np.nan)
        rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldoMean_ult{n+1}meses' for c in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()
        
        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.median().unstack(level = 1, fill_value=np.nan)
        rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldoMedian_ult{n+1}meses' for c in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()
        
#     print(f'haciendo aggregate de {i}')
#     rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby('key_value').agg(agg_rcc)
#     rcc_train_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_train_agg.columns]
#     list_rcc_train_agg.append(rcc_train_agg)
#     gc.collect()
        
rcc_train_ = pd.concat(list_rcc_train_agg, axis=1)
del rcc_train, list_rcc_train_agg

haciendo tipo_credito desde 201703
haciendo RIESGO_DIRECTO desde 201703
haciendo COD_CLASIFICACION_DEUDOR desde 201703
haciendo condicion desde 201703
haciendo cod_instit_financiera desde 201703
haciendo PRODUCTO desde 201703


In [10]:
list_rcc_test_agg = []
for n,i in enumerate([sorted(set(rcc_test.codmes),reverse=True)[-1]]):
    for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR','condicion','cod_instit_financiera','PRODUCTO']:
        print(f'haciendo {c} desde {i}')
        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=np.nan)
        rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldoSum_ult{n+1}meses' for c in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.size().unstack(level = 1, fill_value=np.nan)
        rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldoUnique_ult{n+1}meses' for c in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.min().unstack(level = 1, fill_value=np.nan)
        rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldoMin_ult{n+1}meses' for c in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.max().unstack(level = 1, fill_value=np.nan)
        rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldoMax_ult{n+1}meses' for c in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.std().unstack(level = 1, fill_value=np.nan)
        rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldoStd_ult{n+1}meses' for c in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.mean().unstack(level = 1, fill_value=np.nan)
        rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldoMean_ult{n+1}meses' for c in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.median().unstack(level = 1, fill_value=np.nan)
        rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldoMedian_ult{n+1}meses' for c in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

        
#     print(f'haciendo aggregate de {i}')
#     rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby('key_value').agg(agg_rcc)
#     rcc_test_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_test_agg.columns]
#     list_rcc_test_agg.append(rcc_test_agg)
#     gc.collect()

rcc_test_ = pd.concat(list_rcc_test_agg, axis=1)
del rcc_test, list_rcc_test_agg

haciendo tipo_credito desde 201803
haciendo RIESGO_DIRECTO desde 201803
haciendo COD_CLASIFICACION_DEUDOR desde 201803
haciendo condicion desde 201803
haciendo cod_instit_financiera desde 201803
haciendo PRODUCTO desde 201803


In [11]:
### asegurar que las columnas esten en ambas bases (train/test)
print(rcc_train_.shape, rcc_test_.shape)
keep_columns = list(set(rcc_train_.columns).intersection(rcc_test_.columns))
rcc_train_ = rcc_train_[keep_columns].copy()
rcc_test_ = rcc_test_[keep_columns].copy()
print(rcc_train_.shape, rcc_test_.shape)

(358487, 1260) (396666, 1169)
(358487, 1162) (396666, 1162)


In [12]:
### unir RCC en la base final
train = rcc_train_.copy()
test = rcc_test_.copy()
del rcc_train_, rcc_test_

In [13]:
# train.to_pickle('train_2532features.pkl')
# test.to_pickle('test_2532features.pkl')

In [14]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

def cross_validation_lightgbm_fi(train, y_train, test):
    folds = [train.index[t] for t, v in KFold(5).split(train)]
    test_probs = []
    train_probs = []
    fi = []
    for i, idx in enumerate(folds):
        print("*"*10, i, "*"*10)
        Xt = train.loc[idx]
        yt = y_train.loc[Xt.index].target

        Xv = train.drop(Xt.index)
        yv = y_train.loc[Xv.index].target

        learner = LGBMClassifier(n_estimators=1000, boosting_type='gbdt',min_child_samples=1500, 
                       colsample_bytree=0.8,subsample=0.8, max_bin=200, learning_rate=0.1)
        learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                    eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
        test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
        train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
        fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

    test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
    train_probs = pd.concat(train_probs)
    fi = pd.concat(fi, axis=1).mean(axis=1)
    print("*" * 21)
    print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
    print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))])) 
    return fi

In [15]:
# eliminar variables con zero_importance
while True:
    fi = cross_validation_lightgbm_fi(train, y_train, test)
    zero_importance = fi[fi==0]
    aux = fi[fi>0].sort_values(ascending=False)
    keep_columns = []
    count = 0
    for feature,values in zip(aux.index, aux.values):
        count+=values
        if count<=0.99:
            keep_columns.append(feature)
            
    print(f'total de variables : {len(train.columns)}')
    print(f'variables con importancia acumulada al 99% : {len(keep_columns)}')
    print(f'variables con zero importancia : {len(zero_importance)}')
    train = train[keep_columns]
    test = test[keep_columns]
    if len(zero_importance)==0:
        break

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.829378	training's binary_logloss: 0.316517	valid_1's auc: 0.82109	valid_1's binary_logloss: 0.323158
[100]	training's auc: 0.841328	training's binary_logloss: 0.306597	valid_1's auc: 0.828142	valid_1's binary_logloss: 0.317272
[150]	training's auc: 0.848351	training's binary_logloss: 0.301041	valid_1's auc: 0.830172	valid_1's binary_logloss: 0.315431
[200]	training's auc: 0.854194	training's binary_logloss: 0.296586	valid_1's auc: 0.83133	valid_1's binary_logloss: 0.3146
[250]	training's auc: 0.859372	training's binary_logloss: 0.29266	valid_1's auc: 0.831925	valid_1's binary_logloss: 0.314009
Early stopping, best iteration is:
[252]	training's auc: 0.8596	training's binary_logloss: 0.292498	valid_1's auc: 0.831959	valid_1's binary_logloss: 0.313973
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.829147	training's binary

Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.8313	training's binary_logloss: 0.316413	valid_1's auc: 0.814154	valid_1's binary_logloss: 0.322875
[100]	training's auc: 0.843139	training's binary_logloss: 0.306303	valid_1's auc: 0.820059	valid_1's binary_logloss: 0.31772
[150]	training's auc: 0.850405	training's binary_logloss: 0.30051	valid_1's auc: 0.821994	valid_1's binary_logloss: 0.315949
[200]	training's auc: 0.856253	training's binary_logloss: 0.295976	valid_1's auc: 0.822703	valid_1's binary_logloss: 0.315184
Early stopping, best iteration is:
[200]	training's auc: 0.856253	training's binary_logloss: 0.295976	valid_1's auc: 0.822703	valid_1's binary_logloss: 0.315184
*********************
roc auc estimado:  0.8308611236296664
roc auc varianza:  0.0010187308154542688
total de variables : 480
variables con importancia acumulada al 99% : 442
variables con zero importancia : 0


In [16]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index].target

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index].target

    learner = LGBMClassifier(n_estimators=1000, boosting_type='gbdt',min_child_samples=1500, 
                   colsample_bytree=0.8,subsample=0.8, max_bin=200, learning_rate=0.1)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)
print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))])) 

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.829383	training's binary_logloss: 0.316445	valid_1's auc: 0.820721	valid_1's binary_logloss: 0.323438
[100]	training's auc: 0.841156	training's binary_logloss: 0.306642	valid_1's auc: 0.827559	valid_1's binary_logloss: 0.317697
[150]	training's auc: 0.848379	training's binary_logloss: 0.300885	valid_1's auc: 0.829874	valid_1's binary_logloss: 0.315627
[200]	training's auc: 0.854155	training's binary_logloss: 0.296491	valid_1's auc: 0.830872	valid_1's binary_logloss: 0.314815
[250]	training's auc: 0.859248	training's binary_logloss: 0.292603	valid_1's auc: 0.831842	valid_1's binary_logloss: 0.314099
Early stopping, best iteration is:
[271]	training's auc: 0.861317	training's binary_logloss: 0.291075	valid_1's auc: 0.831944	valid_1's binary_logloss: 0.313999
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.82923	training's 

In [17]:
# test_probs.name = 'target'
# test_probs.to_csv('../results/lightgbm_with_1064features_0.83955.csv') ### score de 0.84242 en la tabla publica

In [18]:
# train.to_pickle('train_1064features.pkl')
# test.to_pickle('test_1064features.pkl')

## Procesar SE

In [19]:
se_train = pd.read_csv(f'{path}/se_train.csv', index_col = 'key_value')
se_test = pd.read_csv(f'{path}/se_test.csv', index_col = 'key_value')

In [20]:
dict_ = {'sexo':'int32',
         'est_cvl':'int32',
         'sit_lab':'int32',
         'cod_ocu':'int32',
         'ctd_hijos':'int32',
         'flg_sin_email':'int32',
         'ctd_veh':'int32',
         'lgr_vot':'int32',
         'prv':'int32',
         'dto':'int32',
         'rgn':'int32',
         'tip_lvledu':'int32'}
se_train = se_train.astype(dict_)
se_test = se_test.astype(dict_)

In [21]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

def cross_validation_lightgbm(train, test, y_train):
    folds = [train.index[t] for t, v in KFold(5).split(train)]
    test_probs = []
    train_probs = []
    fi = []
    for i, idx in enumerate(folds):
        print("*"*10, i, "*"*10)
        Xt = train.loc[idx]
        yt = y_train.loc[Xt.index].target

        Xv = train.drop(Xt.index)
        yv = y_train.loc[Xv.index].target

        learner = LGBMClassifier(n_estimators=1000, boosting_type='gbdt',min_child_samples=1500, 
                       colsample_bytree=0.8,subsample=0.8, max_bin=200, learning_rate=0.1)
        learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                    eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
        test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
        train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
        fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
        gc.collect()
    test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
    train_probs = pd.concat(train_probs)
    fi = pd.concat(fi, axis=1).mean(axis=1)
    print("*" * 21)
    print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
    print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))]))
    return [test_probs, fi]

In [22]:
def get_one_hot(df, feature):
    one_hot = pd.get_dummies(df[feature])
    one_hot.columns = [feature+'_'+str(i) for i in one_hot.columns]
    return one_hot
one_hot_sexo_train = get_one_hot(se_train, 'sexo')
one_hot_sexo_test = get_one_hot(se_test, 'sexo')
one_hot_est_cvl_train = get_one_hot(se_train, 'est_cvl')
one_hot_est_cvl_test = get_one_hot(se_test, 'est_cvl')
one_hot_rgn_train = get_one_hot(se_train, 'rgn')
one_hot_rgn_test = get_one_hot(se_test, 'rgn')

In [23]:
train = train.join(one_hot_sexo_train).join(one_hot_est_cvl_train).join(one_hot_rgn_train).join(se_train[['edad','ctd_veh']])
test = test.join(one_hot_sexo_test).join(one_hot_est_cvl_test).join(one_hot_rgn_test).join(se_test[['edad','ctd_veh']])

In [24]:
[test_probs,fi] = cross_validation_lightgbm(train, test, y_train)

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.832648	training's binary_logloss: 0.315131	valid_1's auc: 0.824697	valid_1's binary_logloss: 0.321771
[100]	training's auc: 0.845558	training's binary_logloss: 0.304235	valid_1's auc: 0.832802	valid_1's binary_logloss: 0.314954
[150]	training's auc: 0.85316	training's binary_logloss: 0.298127	valid_1's auc: 0.835316	valid_1's binary_logloss: 0.312791
[200]	training's auc: 0.859088	training's binary_logloss: 0.29347	valid_1's auc: 0.836466	valid_1's binary_logloss: 0.311792
[250]	training's auc: 0.864298	training's binary_logloss: 0.289463	valid_1's auc: 0.837017	valid_1's binary_logloss: 0.311331
[300]	training's auc: 0.86894	training's binary_logloss: 0.285817	valid_1's auc: 0.837455	valid_1's binary_logloss: 0.310912
Early stopping, best iteration is:
[311]	training's auc: 0.869891	training's binary_logloss: 0.285058	valid_1's auc: 0.837511	valid_1's binary_logloss: 0.31087
***

In [25]:
# test_probs.name = 'target'
# test_probs.to_csv('../results/test_withRccSe_0.84422.csv') #### score de 0.84738 en la tabla publica

## Procesar SUNAT

In [26]:
sunat_train = pd.read_csv(f'{path}/sunat_train.csv')
sunat_test = pd.read_csv(f'{path}/sunat_test.csv')

In [27]:
### eliminar registros duplicados
sunat_train.drop_duplicates(inplace=True)
sunat_test.drop_duplicates(inplace=True)
sunat_train.shape, sunat_test.shape

((292479, 18), (318821, 18))

In [28]:
dict_ = {'tipcontribuyente': 'int32',
         'tippersona': 'int32',
         'ciiu': 'int32',
         'ubigeo': 'int32',
         'condiciondomicilio': 'int32',
         'estadocontribuyente': 'int32',
         'codvia': 'int32',
         'codzona': 'int32',
         'contabilidad': 'int32',
         'facturacion': 'int32',
         'domiciliado': 'int32',
         'comercioexterior': 'int32',
         'cargorele': 'int32',
         'codentidadtributo': 'int32',
         'estadotributo': 'int32'}
sunat_train = sunat_train.astype(dict_)
sunat_test = sunat_test.astype(dict_)

In [29]:
sunat_train['diff_fech'] = sunat_train['fecbaja'] - sunat_train['fecalta']
sunat_test['diff_fech'] = sunat_test['fecbaja'] - sunat_test['fecalta']

In [30]:
moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_sunat = {
            'tipcontribuyente':['nunique',moda],
           'tippersona':['nunique',moda],
           'ciiu':['nunique', moda],
           'ubigeo':['nunique',moda],
           'condiciondomicilio':['nunique',moda],
#            'estadocontribuyente':['nunique',moda],
           'codvia':['nunique',moda],
           'codzona':['nunique',moda],
           'contabilidad':['nunique',moda],
           'facturacion':['nunique',moda],
           'domiciliado':['nunique',moda],
           'comercioexterior':['nunique',moda],
           'cargorele':['nunique',moda],
           'codentidadtributo':['nunique',moda],
           'estadotributo':['nunique',moda],
           'fecalta':['mean','max', 'nunique','min','std'],
           'fecbaja':['mean','max', 'nunique','min'], 
           'diff_fech':['mean','max', 'nunique','min'], 
            }
sunat_train_ = sunat_train.groupby('key_value').agg(agg_sunat)
sunat_train_.columns = [i+'_'+j for i,j in sunat_train_.columns]
sunat_test_ = sunat_test.groupby('key_value').agg(agg_sunat)
sunat_test_.columns = [i+'_'+j for i,j in sunat_test_.columns]
# del sunat_train, sunat_test

In [31]:
def get_keep_columns(train, test):
    print(train.shape, test.shape)
    keep_columns = list(set(train.columns).intersection(test.columns))
    train = train[keep_columns]
    test = test[keep_columns]
    print(train.shape, test.shape)
    return  train, test
def get_crosstab(df, index, feature):
    df = pd.crosstab(df[index], df[feature])
    df.columns = [f'{df.columns.name}_{i}' for i in df.columns]
    return df

import category_encoders as ce
def get_binec(df, feature):
    X_binenc = df[feature]
    encoder = ce.BinaryEncoder(cols=feature)
    X_binenc = encoder.fit_transform(X_binenc)
    return X_binenc

def get_one_hot(df, feature):
    one_hot = pd.get_dummies(df[feature])
    one_hot.columns = [feature+'_'+str(i) for i in one_hot.columns]
    return one_hot

In [32]:
crosstab_estadocontribuyente_train = get_crosstab(sunat_train, 'key_value', 'estadocontribuyente')
crosstab_estadocontribuyente_test = get_crosstab(sunat_test, 'key_value', 'estadocontribuyente')

In [33]:
train = train.join(crosstab_estadocontribuyente_train).join(sunat_train_)
test = test.join(crosstab_estadocontribuyente_test).join(sunat_test_)
train, test = get_keep_columns(train, test)

(358487, 515) (396666, 514)
(358487, 514) (396666, 514)


In [34]:
[test_probs, fi] = cross_validation_lightgbm(train, test, y_train)

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.833097	training's binary_logloss: 0.315019	valid_1's auc: 0.825629	valid_1's binary_logloss: 0.321546
[100]	training's auc: 0.846698	training's binary_logloss: 0.303599	valid_1's auc: 0.833882	valid_1's binary_logloss: 0.314335
[150]	training's auc: 0.854437	training's binary_logloss: 0.29721	valid_1's auc: 0.836969	valid_1's binary_logloss: 0.311711
[200]	training's auc: 0.860714	training's binary_logloss: 0.292419	valid_1's auc: 0.838159	valid_1's binary_logloss: 0.310612
[250]	training's auc: 0.865888	training's binary_logloss: 0.288216	valid_1's auc: 0.838945	valid_1's binary_logloss: 0.309878
Early stopping, best iteration is:
[273]	training's auc: 0.868066	training's binary_logloss: 0.286486	valid_1's auc: 0.839145	valid_1's binary_logloss: 0.309699
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.833304	training's 

In [35]:
fi.sort_values(ascending=False).head(50)
# cod_instit_financiera_max_ult1mes        
# ubigeo_mode  
# ciiu_mode

PRODUCTO_1_saldoMax_ult1meses                       0.012896
edad                                                0.011619
PRODUCTO_1_saldoStd_ult1meses                       0.010994
PRODUCTO_0_saldoMin_ult1meses                       0.010849
ubigeo_mode                                         0.009853
PRODUCTO_1_saldoMean_ult1meses                      0.008889
RIESGO_DIRECTO_1_saldoMax_ult1meses                 0.008555
RIESGO_DIRECTO_-1_saldoMedian_ult1meses             0.008353
PRODUCTO_6_saldoStd_ult1meses                       0.008007
RIESGO_DIRECTO_1_saldoMin_ult1meses                 0.007845
PRODUCTO_12_saldoUnique_ult1meses                   0.007129
PRODUCTO_6_saldoMean_ult1meses                      0.007069
tipo_credito_12_saldoMin_ult1meses                  0.006959
PRODUCTO_3_saldoMin_ult1meses                       0.006913
PRODUCTO_0_saldoStd_ult1meses                       0.006895
PRODUCTO_0_saldoMax_ult1meses                       0.006892
PRODUCTO_1_saldoMedian_u

In [38]:
zero_importance = fi[fi==0]
aux = fi[fi>0].sort_values(ascending=False)
keep_columns = []
count = 0
for feature,values in zip(aux.index, aux.values):
    count+=values
    if count<=0.99:
        keep_columns.append(feature)

print(f'total de variables : {len(train.columns)}')
print(f'variables con importancia acumulada al 99% : {len(keep_columns)}')
print(f'variables con zero importancia : {len(zero_importance)}')

total de variables : 514
variables con importancia acumulada al 99% : 453
variables con zero importancia : 18


In [36]:
test_probs.name = 'target'
test_probs.to_csv('../results/lightgbm_with_514features_0.83731.csv') ### score de  0.83875 en la tabla publica

In [45]:
# train.to_pickle('../data/train_score_0.84862.pkl')
# test.to_pickle('../data/test_score_0.84862.pkl')

## Uniendo ambas bases

In [40]:
train.shape, test.shape

((358487, 514), (396666, 514))

In [60]:
import re
str_ = " ".join(list(train.columns))
cod_instit_financiera_features = re.findall(r'cod_instit_financiera\w+',str_)    
PRODUCTO_features = re.findall(r'PRODUCTO\w+',str_)    
len(cod_instit_financiera_features), len(PRODUCTO_features)

(185, 115)

In [63]:
train = train[cod_instit_financiera_features+PRODUCTO_features]
test = test[cod_instit_financiera_features+PRODUCTO_features]

In [64]:
train_ = pd.read_pickle('../data/train_score_0.84862.pkl')
test_ = pd.read_pickle('../data/test_score_0.84862.pkl')

In [65]:
import re
str_ = " ".join(list(train_.columns))
cod_instit_financiera_features_ = re.findall(r'cod_instit_financiera\w+',str_)    
PRODUCTO_features_ = re.findall(r'PRODUCTO\w+',str_)    
len(cod_instit_financiera_features_), len(PRODUCTO_features_)

(47, 41)

In [68]:
train_ = train_.drop(cod_instit_financiera_features_+PRODUCTO_features_, axis=1)
test_ = test_.drop(cod_instit_financiera_features_+PRODUCTO_features_, axis=1)

In [71]:
train = train.join(train_)
test = test.join(test_)
del train_, test_
train.shape, test.shape

((358487, 1348), (396666, 1348))

In [72]:
[test_probs, fi] = cross_validation_lightgbm(train, test, y_train)

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.849107	training's binary_logloss: 0.298862	valid_1's auc: 0.839495	valid_1's binary_logloss: 0.306045
[100]	training's auc: 0.862473	training's binary_logloss: 0.287593	valid_1's auc: 0.847583	valid_1's binary_logloss: 0.299486
[150]	training's auc: 0.870586	training's binary_logloss: 0.280966	valid_1's auc: 0.850054	valid_1's binary_logloss: 0.297472
[200]	training's auc: 0.877046	training's binary_logloss: 0.275789	valid_1's auc: 0.851135	valid_1's binary_logloss: 0.29661
[250]	training's auc: 0.882699	training's binary_logloss: 0.271229	valid_1's auc: 0.851433	valid_1's binary_logloss: 0.296276
Early stopping, best iteration is:
[281]	training's auc: 0.885817	training's binary_logloss: 0.268606	valid_1's auc: 0.851718	valid_1's binary_logloss: 0.296058
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.849306	training's 

In [85]:
fi.sort_values(ascending=False).head(50)

RIESGO_DIRECTO_1_saldoUnique_ult1meses               0.010687
edad                                                 0.009224
PRODUCTO_1_saldoStd_ult1meses                        0.008240
PRODUCTO_1_saldoMax_ult1meses                        0.008136
ubigeo_mode                                          0.007619
PRODUCTO_3_saldoUnique_ult1meses                     0.006920
RIESGO_DIRECTO_1_saldoStd_ult1meses                  0.006619
PRODUCTO_1_saldoMean_ult1meses                       0.005491
RIESGO_DIRECTO_-1_saldoMedian_ult1meses              0.005416
PRODUCTO_8_saldoSum_ult1meses                        0.005237
PRODUCTO_0_saldoMin_ult1meses                        0.004985
RIESGO_DIRECTO_1_saldoMin_ult1meses                  0.004925
RIESGO_DIRECTO_1_saldoUnique_ult12meses              0.004895
PRODUCTO_3_saldoMean_ult1meses                       0.004889
cod_instit_financiera_10_saldoStd_ult1meses          0.004738
RIESGO_DIRECTO_1_saldoMedian_ult2meses               0.004733
PRODUCTO

## Reduciendo Variables

In [86]:
zero_importance = fi[fi==0]
aux = fi[fi>0].sort_values(ascending=False)
keep_columns = []
count = 0
for feature,values in zip(aux.index, aux.values):
    count+=values
    if count<=0.99:
        keep_columns.append(feature)

print(f'total de variables : {len(fi)}')
print(f'variables con importancia acumulada al 99% : {len(keep_columns)}')
print(f'variables con zero importancia : {len(zero_importance)}')

total de variables : 1348
variables con importancia acumulada al 99% : 1212
variables con zero importancia : 29


In [75]:
test_probs.name = 'target'
test_probs.to_csv('../results/lightgbm_with_1348features_0.85109.csv') ### score de  0.85375 en la tabla publica

In [87]:
[test_probs_, fi_] = cross_validation_lightgbm(train[keep_columns], test[keep_columns], y_train)

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.849111	training's binary_logloss: 0.298938	valid_1's auc: 0.83967	valid_1's binary_logloss: 0.306061
[100]	training's auc: 0.862629	training's binary_logloss: 0.287595	valid_1's auc: 0.848015	valid_1's binary_logloss: 0.299491
[150]	training's auc: 0.870694	training's binary_logloss: 0.281029	valid_1's auc: 0.850591	valid_1's binary_logloss: 0.297392
Early stopping, best iteration is:
[170]	training's auc: 0.873462	training's binary_logloss: 0.278826	valid_1's auc: 0.851116	valid_1's binary_logloss: 0.296981
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.848988	training's binary_logloss: 0.299077	valid_1's auc: 0.840879	valid_1's binary_logloss: 0.305056
[100]	training's auc: 0.862524	training's binary_logloss: 0.287676	valid_1's auc: 0.848632	valid_1's binary_logloss: 0.298487
[150]	training's auc: 0.870531	training's 

In [88]:
zero_importance = fi_[fi_==0]
aux = fi_[fi_>0].sort_values(ascending=False)
keep_columns = []
count = 0
for feature,values in zip(aux.index, aux.values):
    count+=values
    if count<=0.99:
        keep_columns.append(feature)

print(f'total de variables : {len(fi_)}')
print(f'variables con importancia acumulada al 99% : {len(keep_columns)}')
print(f'variables con zero importancia : {len(zero_importance)}')

total de variables : 1212
variables con importancia acumulada al 99% : 1134
variables con zero importancia : 1


In [89]:
test_probs_.name = 'target'
test_probs_.to_csv('../results/lightgbm_with_1212features_0.85095.csv') ### score de  0.85367 en la tabla publica

In [90]:
[test_probs_, fi_] = cross_validation_lightgbm(train[keep_columns], test[keep_columns], y_train)

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.849287	training's binary_logloss: 0.298893	valid_1's auc: 0.840366	valid_1's binary_logloss: 0.305615
[100]	training's auc: 0.862652	training's binary_logloss: 0.287612	valid_1's auc: 0.848206	valid_1's binary_logloss: 0.299179
[150]	training's auc: 0.87067	training's binary_logloss: 0.281108	valid_1's auc: 0.85059	valid_1's binary_logloss: 0.297144
[200]	training's auc: 0.877032	training's binary_logloss: 0.275889	valid_1's auc: 0.851831	valid_1's binary_logloss: 0.296178
[250]	training's auc: 0.882567	training's binary_logloss: 0.271339	valid_1's auc: 0.852541	valid_1's binary_logloss: 0.295648
Early stopping, best iteration is:
[252]	training's auc: 0.882778	training's binary_logloss: 0.271161	valid_1's auc: 0.852541	valid_1's binary_logloss: 0.295638
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.848896	training's b

In [92]:
fi_.sort_values(ascending=False).head(50)

RIESGO_DIRECTO_1_saldoUnique_ult1meses               0.011269
edad                                                 0.008779
PRODUCTO_1_saldoStd_ult1meses                        0.007905
PRODUCTO_1_saldoMax_ult1meses                        0.007871
ubigeo_mode                                          0.007420
PRODUCTO_3_saldoUnique_ult1meses                     0.006676
RIESGO_DIRECTO_1_saldoStd_ult1meses                  0.006310
PRODUCTO_0_saldoMin_ult1meses                        0.005592
RIESGO_DIRECTO_-1_saldoMedian_ult1meses              0.005316
sexo_0                                               0.005121
PRODUCTO_1_saldoMean_ult1meses                       0.005081
cod_instit_financiera_10_saldoStd_ult1meses          0.005060
PRODUCTO_8_saldoUnique_ult1meses                     0.004964
RIESGO_DIRECTO_1_saldoMin_ult1meses                  0.004935
RIESGO_DIRECTO_1_saldoMedian_ult1meses               0.004826
PRODUCTO_8_saldoSum_ult1meses                        0.004726
RIESGO_D

In [91]:
zero_importance = fi_[fi_==0]
aux = fi_[fi_>0].sort_values(ascending=False)
keep_columns = []
count = 0
for feature,values in zip(aux.index, aux.values):
    count+=values
    if count<=0.99:
        keep_columns.append(feature)

print(f'total de variables : {len(fi_)}')
print(f'variables con importancia acumulada al 99% : {len(keep_columns)}')
print(f'variables con zero importancia : {len(zero_importance)}')

total de variables : 1134
variables con importancia acumulada al 99% : 1072
variables con zero importancia : 0


In [94]:
test_probs_.name = 'target'
test_probs_.to_csv('../results/lightgbm_with_1134features_0.85133.csv') ### score de  0.85379 en la tabla publica

In [97]:
[test_probs_, fi_] = cross_validation_lightgbm(train[keep_columns], test[keep_columns], y_train)

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.849192	training's binary_logloss: 0.298944	valid_1's auc: 0.840273	valid_1's binary_logloss: 0.305633
[100]	training's auc: 0.862318	training's binary_logloss: 0.287691	valid_1's auc: 0.847924	valid_1's binary_logloss: 0.299277
[150]	training's auc: 0.870461	training's binary_logloss: 0.281069	valid_1's auc: 0.850561	valid_1's binary_logloss: 0.297056
[200]	training's auc: 0.876888	training's binary_logloss: 0.275894	valid_1's auc: 0.851653	valid_1's binary_logloss: 0.296246
[250]	training's auc: 0.882611	training's binary_logloss: 0.271388	valid_1's auc: 0.852212	valid_1's binary_logloss: 0.295765
Early stopping, best iteration is:
[283]	training's auc: 0.886039	training's binary_logloss: 0.268603	valid_1's auc: 0.852394	valid_1's binary_logloss: 0.295624
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.849263	training's

In [98]:
test_probs_.name = 'target'
test_probs_.to_csv('../results/lightgbm_with_1072features_0.85127.csv') ### score de  0.85381  en la tabla publica

In [102]:
train = train[keep_columns]
test = test[keep_columns]
train.shape, test.shape

((358487, 1072), (396666, 1072))

In [103]:
train.to_pickle('../data/train_1072features_score_0.84862.pkl')
test.to_pickle('../data/test_1072features_score_0.84862.pkl')